# Download data files:
Uploading files to Colab failed (200MB upload cap), so get them from my nextcloud:

# Define a Dataset: 


In [1]:
import numpy as np
import h5py
import torch
import torchmetrics
import torch.nn.functional as F
import pytorch_lightning as pl


from torch.utils.data import Dataset, DataLoader
from torch.nn import BatchNorm1d, Linear, ReLU, Sequential, MSELoss
from torch_geometric.nn import DynamicEdgeConv, global_mean_pool
from torch_geometric.data import Batch



In [4]:
with h5py.File('../project_data/small_set_1_train_0_shuffled.h5', "r") as f:
    x_features=[f['x'].attrs[key] for key in f['x'].attrs.keys()]
    print('Available features:',x_features)
    print()
    print('Shape data:',f['x'].shape)
    sample_x=f['x'][:8,:,:]
    sample_y=f['y'][:8]


Available features: [b'EARRAY', 0, b'nodes', b'1.1', 'channel_id', 'dir_x', 'pos_z', 't0', 'time', 'tot', 'triggered', 'is_valid', 'dir_y', 'dir_z', 'dom_id', 'du', 'floor', 'group_id', 'pos_x', 'pos_y']

Shape data: (410975, 2000, 16)


In [5]:

{key:sample_y[0][key] for key in sample_y[0].dtype.names}

{'event_id': 4761,
 'particle_type': -13.0,
 'energy': 362.335,
 'is_cc': 0.0,
 'bjorkeny': 0.0,
 'dir_x': -0.253578,
 'dir_y': 0.706406,
 'dir_z': -0.660824,
 'time_interaction': 35905258.923828125,
 'run_id': 5788.0,
 'vertex_pos_x': 71.628,
 'vertex_pos_y': -293.2989838709678,
 'vertex_pos_z': 333.284,
 'n_hits': 68.0,
 'weight_w1': 10.0,
 'weight_w2': 10.0,
 'weight_w3': 10.0,
 'n_gen': 1.0,
 'prod_identifier': 2.0,
 'std_dir_x': -0.2714447595320438,
 'std_dir_y': 0.7792974171365362,
 'std_dir_z': -0.5648126044688755,
 'std_beta0': 0.09251125156879425,
 'std_lik': 40.80266540149382,
 'std_n_hits_gandalf': 32.0,
 'std_pos_x': -24.575529242829962,
 'std_pos_y': -35.57214054543206,
 'std_pos_z': 90.95574060192375,
 'std_energy': 143.94716798722442,
 'std_lik_energy': 33.15291976928711,
 'std_length': 10.79861831665039,
 'group_id': 35782}

In [6]:
sample_x[0,0:10,11:14]

array([[ 2.0710222e+05,  1.6976100e+02,  3.0000000e+01],
       [ 2.0715936e+05,  1.6424001e+02,  2.6000000e+01],
       [ 2.0721278e+05, -2.2567101e+02,  2.4000000e+01],
       [ 2.0726403e+05, -6.7526599e+02,  3.1000000e+01],
       [ 2.0726369e+05, -4.6661200e+02,  3.1000000e+01],
       [ 2.0726497e+05,  7.2000000e+01,  3.3000000e+01],
       [ 2.0732053e+05, -1.6743800e+02,  3.3000000e+01],
       [ 2.0731942e+05,  5.5778000e+01,  2.3000000e+01],
       [ 2.0737469e+05,  4.3570999e+01,  2.6000000e+01],
       [ 2.0737392e+05,  4.0754002e+01,  1.6000000e+01]], dtype=float32)

In [49]:
class HDF5Dataset(Dataset):
    def __init__(self, path, features=["pos_x", "pos_y", "pos_z", "time", "dir_x", "dir_y", "dir_z"], y_feature="particle_type", particle_type=13.0, batch_size=64):
        """ Loads the data from the hdf5 format provided by OrcaSong and converts it to data that can be used by PyTorch
        
        Args:
            path (str): path to the dataset
            features (list[str]): List of features to select from the event data and use as input features
            y_feature  (str): Output feature to select
            particle_type (None or float):  ID of the particle you want to classify, it will be label 0 and all else will be label 1.
                                            Must be None when y_feature is not `particle_type`
            batch_size (int): number of samples in mini batch
        Examples:
            Electron vs Background classification (default):
            ```
                HDF5Dataset("pathtodata.h5", y_feature="particle_type", particle_type=13.0)
            ```
            Energy regression with only xyzct:
            ```
                HDF5Dataset("pathtodata.h5", features=["pos_x", "pos_y", "pos_z", "time"], y_feature="energy", particle_type=None)
            ```

        Lookup of table for particle_type of Leptons:
          electron          | 11
          electron neutrino | 12
          muon              | 13
          muon neutrino     | 14
          tau               | 15
          tau neutrino      | 16
        Antiparticle is the same as particle but with minus sign
        Source: https://pdg.lbl.gov/2007/reviews/montecarlorpp.pdf
        """
        with h5py.File(path, "r") as f:
            self.groups = list(dict(f).keys())
            self.length = len(f["y"]) // batch_size + 1
            self._max_index = len(f["y"])
            print("The available y features are: ", f["y"][0].dtype.names)
        self.filename = path
        if y_feature!="particle_type":
            assert particle_type==None, "Selected a y_feature other than 'particle_type' and specified some value for particle_type as argument, which must be None for non particle_type output feature."
        self.y_feature = y_feature
        self.particle_type = particle_type
        self.batch_size = batch_size
        self._cache_x_column_names()
        self.x_mask = self.init_x_mask(features)


    def _cache_x_column_names(self):
        """Cache which columns are available in the features

        Raises:
            ValueError: It failed to read the hit_info columns
        """
        try:
            with h5py.File(self.filename, "r") as f:
                self.x_feature_dict = {
                    f["x"].attrs[f"hit_info_{i}"]: i for i in range(f["x"].shape[-1])
                }
            print("cached the following x input features", self.x_feature_dict)
        except Exception:
            raise ValueError("Can not read column names from dataset attributes")

    def init_x_mask(self, features):
        """Compute a mask that is used to select the feature columns from the data

        Args:
            features (list[str]): list of features present to load

        Returns:
            np.array: selection of column index from the features to use
        """
        x_mask = [self.x_feature_dict[feat] for feat in features]
        return np.array(x_mask)

    def __getitem__(self, index):
        """Get an sample from the h5 dataset
        x contains: (x,y,z,ct, dir_x, dir_y, dir_z)
        y contains a label 

        Args:
            index (int): index of the batch

        Returns:
            x (torch.Tensor): Tensor with the x data (for each of the vertices)
            y (torch.Tensor): Tensor with the y data (for the graph)
            batch_idx (torch.Tensor): Tensor that assigns the right batch index to each x point
        """
        with h5py.File(self.filename, "r") as f:
            index = slice(index * self.batch_size, min(self._max_index,(index + 1) * self.batch_size))
            
            x = f["x"][index]
            lengths = (np.sum(x[:, :, -1:], axis=1)).astype(int)
            batch_idx = np.hstack(
                  [
                      np.ones(length) * batch_idx
                      for batch_idx, length in enumerate(lengths)
                  ]
              )
            x = x[x[:, :, -1] == 1][:, self.x_mask]
            y = f["y"][index][self.y_feature]
            if self.y_feature=="particle_type" and self.particle_type:
                y = torch.LongTensor(~(abs(y) == self.particle_type))
            else:
                y = torch.Tensor(y.copy())#.type(torch.int32)
        return x, y, torch.LongTensor(batch_idx)

    def __len__(self):
        return self.length

In [50]:
path='small_set_1_train_0_shuffled.h5'
dataset = HDF5Dataset(path,features=["pos_x", "pos_y", "pos_z", "tot", "dir_x", "dir_y", "dir_z"], y_feature="energy", particle_type=None, batch_size=16)

The available y features are:  ('event_id', 'particle_type', 'energy', 'is_cc', 'bjorkeny', 'dir_x', 'dir_y', 'dir_z', 'time_interaction', 'run_id', 'vertex_pos_x', 'vertex_pos_y', 'vertex_pos_z', 'n_hits', 'weight_w1', 'weight_w2', 'weight_w3', 'n_gen', 'prod_identifier', 'std_dir_x', 'std_dir_y', 'std_dir_z', 'std_beta0', 'std_lik', 'std_n_hits_gandalf', 'std_pos_x', 'std_pos_y', 'std_pos_z', 'std_energy', 'std_lik_energy', 'std_length', 'group_id')
cached the following x input features {'channel_id': 0, 'dir_x': 1, 'dir_y': 2, 'dir_z': 3, 'dom_id': 4, 'du': 5, 'floor': 6, 'group_id': 7, 'pos_x': 8, 'pos_y': 9, 'pos_z': 10, 't0': 11, 'time': 12, 'tot': 13, 'triggered': 14, 'is_valid': 15}


In [51]:
x,y,_=dataset.__getitem__(1)
y

tensor([3.5538e+02, 1.0633e+05, 1.8340e+02, 1.2972e+03, 2.0516e+03, 5.8543e+01,
        2.4592e+05, 3.0711e+02, 2.2140e+01, 2.1608e+00, 1.4521e+02, 2.5784e+03,
        1.9415e+02, 1.5436e+02, 8.2658e+01, 4.3011e+00])

# Define a network architecture

In [62]:
class DECNetwork(pl.LightningModule):
    def __init__(self, batchnorm_kwargs=None, conf=None):
        """Dynamic EdgeConvolution Network https://arxiv.org/abs/1801.07829 with
           the dynamic KNN computation as presented in https://arxiv.org/abs/1902.08570 """
        super().__init__()
        ## Lightning configuration
        #self.accuracy = torchmetrics.Accuracy()
        #self.precision = torchmetrics.Precision()
        #self.confusionmatrix = torchmetrics.ConfusionMatrix(1)

        ## Defining the Network Architecture
        nn = Sequential(
            Linear(2 * 7, 64),
            BatchNorm1d(64, **batchnorm_kwargs),
            ReLU(),
            Linear(64, 64),
            BatchNorm1d(64, **batchnorm_kwargs),
            ReLU(),
            Linear(64, 64),
            BatchNorm1d(64, **batchnorm_kwargs),
            ReLU(),
        )
        self.edge_1 = DynamicEdgeConv(nn, aggr="mean", k=32)
        nn = Sequential(
            Linear(128, 128),
            BatchNorm1d(128, **batchnorm_kwargs),
            ReLU(),
            Linear(128, 128),
            BatchNorm1d(128, **batchnorm_kwargs),
            ReLU(),
            Linear(128, 128),
            BatchNorm1d(128, **batchnorm_kwargs),
            ReLU(),
        )
        self.edge_2 = DynamicEdgeConv(nn, aggr="mean", k=32)
        nn = Sequential(
            Linear(256, 256),
            BatchNorm1d(256, **batchnorm_kwargs),
            ReLU(),
            Linear(256, 256),
            BatchNorm1d(256, **batchnorm_kwargs),
            ReLU(),
            Linear(256, 256),
            BatchNorm1d(256, **batchnorm_kwargs),
            ReLU(),
        )
#        self.edge_3 = DynamicEdgeConv(nn, aggr="mean", k=32)
        self.shortcut_1 = Sequential(Linear(7, 64), BatchNorm1d(64), ReLU())
        self.shortcut_2 = Sequential(Linear(64, 128), BatchNorm1d(128), ReLU())
#        self.shortcut_3 = Sequential(Linear(128, 256), BatchNorm1d(256), ReLU())
#        self.lin_1 = Linear(256, 256)
        self.lin_2 = Linear(128, 128)
        self.lin_3 = Linear(128, 1)

    def forward(self, data):
        x, batch_idx = data.x, data.batch
        # in lightning, forward defines the prediction/inference actions
        # edgeconv layer 1
        sc = self.shortcut_1(x)
        x = self.edge_1(x, batch_idx)
        x = F.relu(x + sc)
        # edgeconv layer 2
        sc = self.shortcut_2(x)
        x = self.edge_2(x, batch_idx)
        x = F.relu(x + sc)
        # edgeconv layer 3
#        sc = self.shortcut_3(x)
#        x = self.edge_3(x, batch_idx)
#        x = F.relu(x + sc)
        x = global_mean_pool(x, batch=batch_idx)
        # now apply
 #       x = F.relu(self.lin_1(x))
        x = F.relu(self.lin_2(x))
        x = self.lin_3(x)
        return F.relu(x)
        

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop. It is independent of forward
        batch = Batch(
            x=batch[0].squeeze(), y=batch[1].squeeze(), batch=batch[2].squeeze(),
        )
        out = self.forward(batch)
        loss = F.l1_loss(out, batch.y.view(-1,1))
        self.log("train_loss", loss)
        #self.log('train_acc_step', self.accuracy(out, batch.y), on_epoch=False)
        return loss

    def validation_step(self, batch, batch_idx):
        batch = Batch(
            x=batch[0].squeeze(), y=batch[1].squeeze(), batch=batch[2].squeeze(),
        )
        y_hat = self.forward(batch)
        loss = F.l1_loss(y_hat, batch.y.view(-1,1))
        self.log("val_loss", loss)
        #self.log('valid_acc', self.accuracy(y_hat, batch.y), on_step=True, on_epoch=True)
        #self.log('valid_matrix', self.confusionmatrix(y_hat, batch.y), on_epoch=True)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer


# Train the neural network
! Important: since the hdf5 format is much more efficient at handling IO for the data, only set batch_size in the HDF5Dataset and keep the DataLoader batch_size on 1 at all times.

In [63]:
train_data = HDF5Dataset(path,features=["pos_x", "pos_y", "pos_z", "tot", "dir_x", "dir_y", "dir_z"], y_feature="energy", particle_type=None, batch_size=16)

The available y features are:  ('event_id', 'particle_type', 'energy', 'is_cc', 'bjorkeny', 'dir_x', 'dir_y', 'dir_z', 'time_interaction', 'run_id', 'vertex_pos_x', 'vertex_pos_y', 'vertex_pos_z', 'n_hits', 'weight_w1', 'weight_w2', 'weight_w3', 'n_gen', 'prod_identifier', 'std_dir_x', 'std_dir_y', 'std_dir_z', 'std_beta0', 'std_lik', 'std_n_hits_gandalf', 'std_pos_x', 'std_pos_y', 'std_pos_z', 'std_energy', 'std_lik_energy', 'std_length', 'group_id')
cached the following x input features {'channel_id': 0, 'dir_x': 1, 'dir_y': 2, 'dir_z': 3, 'dom_id': 4, 'du': 5, 'floor': 6, 'group_id': 7, 'pos_x': 8, 'pos_y': 9, 'pos_z': 10, 't0': 11, 'time': 12, 'tot': 13, 'triggered': 14, 'is_valid': 15}


In [64]:
#train_data = HDF5Dataset("small_set_1_train_0_shuffled.h5", batch_size=32)
train_loader = DataLoader(
    train_data,
    batch_size=1,
    num_workers=4,
    pin_memory=True,
    shuffle=False,
)

val_data = HDF5Dataset("small_set_1_validate_0_shuffled.h5",features=["pos_x", "pos_y", "pos_z", "tot", "dir_x", "dir_y", "dir_z"], y_feature="energy", particle_type=None, batch_size=32)

val_loader = DataLoader(
    val_data,
    batch_size=1,
    num_workers=4,
    pin_memory=True,
    shuffle=False,
)
model = DECNetwork(batchnorm_kwargs={"eps": 1e-3, "momentum": 1e-1})

The available y features are:  ('event_id', 'particle_type', 'energy', 'is_cc', 'bjorkeny', 'dir_x', 'dir_y', 'dir_z', 'time_interaction', 'run_id', 'vertex_pos_x', 'vertex_pos_y', 'vertex_pos_z', 'n_hits', 'weight_w1', 'weight_w2', 'weight_w3', 'n_gen', 'prod_identifier', 'std_dir_x', 'std_dir_y', 'std_dir_z', 'std_beta0', 'std_lik', 'std_n_hits_gandalf', 'std_pos_x', 'std_pos_y', 'std_pos_z', 'std_energy', 'std_lik_energy', 'std_length', 'group_id')
cached the following x input features {'channel_id': 0, 'dir_x': 1, 'dir_y': 2, 'dir_z': 3, 'dom_id': 4, 'du': 5, 'floor': 6, 'group_id': 7, 'pos_x': 8, 'pos_y': 9, 'pos_z': 10, 't0': 11, 'time': 12, 'tot': 13, 'triggered': 14, 'is_valid': 15}


In [65]:
trainer = pl.Trainer(
    max_epochs=6,
    gpus=1,
    precision=32,
    log_every_n_steps=10,
    progress_bar_refresh_rate=1,
    fast_dev_run=False,
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [66]:
trainer.fit(model, train_loader, val_loader)


  | Name       | Type            | Params
-----------------------------------------------
0 | edge_1     | DynamicEdgeConv | 9.7 K 
1 | edge_2     | DynamicEdgeConv | 50.3 K
2 | shortcut_1 | Sequential      | 640   
3 | shortcut_2 | Sequential      | 8.6 K 
4 | lin_2      | Linear          | 16.5 K
5 | lin_3      | Linear          | 129   
-----------------------------------------------
85.8 K    Trainable params
0         Non-trainable params
85.8 K    Total params


Epoch 0:  91%|█████████ | 25687/28179 [1:13:02<07:05,  5.86it/s, loss=1.68e+04, v_num=22]
Validating: 0it [00:00, ?it/s]
Epoch 1:  91%|█████████ | 25686/28179 [1:14:07<07:11,  5.78it/s, loss=1.68e+04, v_num=22]
Validating: 0it [00:00, ?it/s]
Epoch 2:  91%|█████████ | 25686/28179 [1:14:31<07:14,  5.74it/s, loss=1.68e+04, v_num=22]
Validating: 0it [00:00, ?it/s]
Epoch 3:  91%|█████████ | 25686/28179 [1:14:36<07:14,  5.74it/s, loss=1.67e+04, v_num=22]
Validating: 0it [00:00, ?it/s]
Epoch 4:  91%|█████████ | 25686/28179 [1:14:35<07:14,  5.74it/s, loss=1.66e+04, v_num=22]
Validating: 0it [00:00, ?it/s]
Epoch 5:  91%|█████████ | 25686/28179 [1:14:24<07:13,  5.75it/s, loss=1.66e+04, v_num=22]
Validating: 0it [00:00, ?it/s]
Epoch 5: 100%|██████████| 28179/28179 [1:17:30<00:00,  6.06it/s, loss=1.66e+04, v_num=22]


1

In [ ]:
#trainer.fit(model, train_loader, val_loader)

In [ ]:
trainer.save_checkpoint(trainer.log_dir+"/trained_model.ckpt")